In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the directory above the current notebook to the system path
sys.path.append(os.path.abspath(os.path.join('..')))

In [3]:
from io_images import get_images_infoframe
import paramiko

Get chunk images .tiff

In [4]:
windows = False
folder_path = '/run/user/1001/gvfs/smb-share:server=fs.ista.ac.at,share=drives/tnegrell/archive/siegegrp/AlVe/MORPHOMICS2.0_MICROGLIA_BRAIN_ATLAS'


In [ ]:
# windows = True
# folder_path = r"\\fs.ista.ac.at\drives\aventuri\archive\siegegrp\AlVe\MORPHOMICS2.0_MICROGLIA_BRAIN_ATLAS"


In [6]:
project_path = os.path.join(folder_path, "chunk_images")

infoframe = get_images_infoframe(project_path, 
                                 extension='.tif',
                                 conditions = ['Age', 'Sex', 'Animal', 'Slide'],
                                 windows=windows)

Found 631 files...
There are 631 files in folder_location


In [7]:
infoframe

,Age,Sex,Animal,Slide,file_name,file_path
0,18m,F,Animal_1,Slide_4,microglia_Age_18m_Sex_F_Animal_1_Slide_4_seq_4...,/run/user/1001/gvfs/smb-share:server=fs.ista.a...
1,18m,F,Animal_1,Slide_4,microglia_Age_18m_Sex_F_Animal_1_Slide_4_seq_4...,/run/user/1001/gvfs/smb-share:server=fs.ista.a...
2,18m,F,Animal_1,Slide_4,microglia_Age_18m_Sex_F_Animal_1_Slide_4_seq_3...,/run/user/1001/gvfs/smb-share:server=fs.ista.a...
3,18m,F,Animal_1,Slide_4,microglia_Age_18m_Sex_F_Animal_1_Slide_4_seq_4...,/run/user/1001/gvfs/smb-share:server=fs.ista.a...
4,18m,F,Animal_1,Slide_4,microglia_Age_18m_Sex_F_Animal_1_Slide_4_seq_1...,/run/user/1001/gvfs/smb-share:server=fs.ista.a...
...,...,...,...,...,...,...
626,18m,F,Animal_2,Slide_2,microglia_Age_18m_Sex_F_Animal_2_Slide_2_seq_1...,/run/user/1001/gvfs/smb-share:server=fs.ista.a...
627,18m,F,Animal_2,Slide_2,microglia_Age_18m_Sex_F_Animal_2_Slide_2_seq_0...,/run/user/1001/gvfs/smb-share:server=fs.ista.a...
628,18m,F,Animal_2,Slide_2,microglia_Age_18m_Sex_F_Animal_2_Slide_2_seq_3...,/run/user/1001/gvfs/smb-share:server=fs.ista.a...
629,18m,F,Animal_2,Slide_2,microglia_Age_18m_Sex_F_Animal_2_Slide_2_seq_0...,/run/user/1001/gvfs/smb-share:server=fs.ista.a...


Get one chunk

In [8]:
# Define the file name you are searching for
search_file = "microglia_Age_18m_Sex_F_Animal_1_Slide_0_seq_0_chunk_4_over_5_x__-137695_-127435__y__46205_47957__.tiff"

# Get the index of the row containing the file name
index = infoframe.loc[infoframe['file_name'] == search_file].index

# If you want to print or use the index
print(f"Index of the row: {index}")


Index of the row: Index([], dtype='int64')


Get the one

In [9]:
chunk_example = infoframe.iloc[243]

age = chunk_example['Age']
sex = chunk_example['Sex']
animal = chunk_example['Animal']
slide = chunk_example['Slide']
filename = chunk_example['file_name']

In [10]:
base_path = f'/mnt/archive/archive/siegegrp/AlVe/MORPHOMICS2.0_MICROGLIA_BRAIN_ATLAS'
function = f'CUDA_VISIBLE_DEVICES=0 trAIce img2swc'
linux_folderpath = f' -ip {base_path}/chunk_images/{age}/{sex}/{animal}/{slide}/{filename}' 

cube_params = f' -wss "(128, 128, 16)" -wsb "(128, 128, 16)" -tp "(0.3, 0.3, 1.0)"'
save_folderpath = f' -spd {base_path}/traced_microglia/{age}/{sex}/{animal}/{slide}/{filename} -mp ./ -spsl ./ -nw 1 -bsp ./'


In [11]:
command = function + linux_folderpath + cube_params + save_folderpath

In [12]:
print(command)

CUDA_VISIBLE_DEVICES=0 trAIce img2swc -ip /mnt/archive/archive/siegegrp/AlVe/MORPHOMICS2.0_MICROGLIA_BRAIN_ATLAS/chunk_images/18m/F/Animal_1/Slide_0/microglia_Age_18m_Sex_F_Animal_1_Slide_0_seq_3_chunk_1_over_5_x__-37461_-27220__y__25669_35929__.tif -wss "(128, 128, 16)" -wsb "(128, 128, 16)" -tp "(0.3, 0.3, 1.0)" -spd /mnt/archive/archive/siegegrp/AlVe/MORPHOMICS2.0_MICROGLIA_BRAIN_ATLAS/traced_microglia/18m/F/Animal_1/Slide_0/microglia_Age_18m_Sex_F_Animal_1_Slide_0_seq_3_chunk_1_over_5_x__-37461_-27220__y__25669_35929__.tif -mp ./ -spsl ./ -nw 1 -bsp ./


In [ ]:
# SSH connection details
ssh_user = "tnegrell"
ssh_host = "10.6.46.11"
remote_command = command
password = "123456" 

# Create an SSH client
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

try:
    # Connect to the remote machine with a password
    ssh.connect(ssh_host, username=ssh_user, password=password)

    # Execute the command
    stdin, stdout, stderr = ssh.exec_command(remote_command)

    # Read and print the command's output
    print("Command Output:\n", stdout.read().decode())
    print("Command Error:\n", stderr.read().decode())
finally:
    # Close the SSH connection
    ssh.close()


In [ ]:
import os
import click
from . import preprocess as img_preprocess
from . import model_io
from . import postprocess
import subprocess
import ast
from typing import Union
import time
import numpy as np


@click.group(chain=True)
def cli():
    pass


def img2soma_func(img_path, soma_model_save_path, soma_locs, window_size_soma,
                  overlap, target_px2mu, save_path_dataset,
                  mat_path, dataset_name, num_workers, ignore_flags) -> Union[bool, str]:
    """ Detect soma from the image file and save the results to the save_path

        Args:
            img_path (str): The path to the image file
            soma_model_save_path (str): The path to the soma model
            soma_locs (str): The path to save the segmented somas of soma model
            window_size_soma (tuple): The size of the window for soma detection cubes (soma model)
            overlap (tuple): The overlap of the window for soma detection cubes (soma model)
            target_px2mu (tuple): The target pixel to micron ratio used to scale the image
            save_path_dataset (str): The path to save the cubes for soma detection
            mat_path (str): The path to save the mat file
            dataset_name (str): The name of the dataset
            num_workers (int): The number of workers to use
            soma_model_name (str): The name of the soma detection model in models class
            ignore_flags (bool): Whether to ignore flags. If True, it will run the function regardless of previous computations
        Returns:
            str: The path to the saved soma locations
    """

    # Convert arguments str type to tuple
    window_size_soma = str(window_size_soma)
    overlap = str(overlap)
    target_px2mu = str(target_px2mu)

    window_size_soma = ast.literal_eval(window_size_soma)
    overlap = ast.literal_eval(overlap)
    target_px2mu = ast.literal_eval(target_px2mu)

    # Set dataset name based on imaris file
    if not dataset_name:
        dataset_name = os.path.basename(img_path).split(".")[0]

    # Set paths and flag paths
    datasets_path = os.path.join(save_path_dataset, dataset_name)
    if soma_locs is None:
        soma_locs = os.path.join(datasets_path, 'soma')
    else:
        soma_locs = os.path.join(soma_locs, dataset_name)
    flag_path_data = os.path.join(datasets_path, 'flag_data')
    flag_path_cubes = os.path.join(datasets_path, 'flag_cubes')
    flag_path_mat = os.path.join(mat_path, dataset_name, 'flag_mat')

    # Check if flag is set. If so, terminate
    if (not ignore_flags) and os.path.exists(flag_path_data):
        print("IMG to Soma was done already.")
        return soma_locs

    # Make mat file from img file
    print("=======Make mat files out of img/nd2 file=======")
    print(f"img_path: {img_path}")
    # Check mat flag
    if ignore_flags or (not os.path.exists(flag_path_mat)):
        # Run shell script to make mat file. main.sh handles file types: ims, nd2, tiff
        # result = subprocess.run(
        #     f"./main.sh make_datasets_file \"{img_path}\" \"./ims_reader\" \"{mat_path}\"", shell=True)
            
        # # Check if error in reading data
        # if result.returncode != 0:
        #     print("See error above. Error in reading data.")
        #     return False

        # Check file type
        if img_path.endswith('.ims'):
            data = img_preprocess.make_dataset_ims(img_path, mat_path)
        elif img_path.endswith('.nd2'):
            data = img_preprocess.make_dataset_nd2(img_path, mat_path)
        elif img_path.endswith('.tif'):
            data = img_preprocess.make_dataset_tif(img_path, mat_path)

        # Set flag for creating mat file
        open(flag_path_mat, 'w').close()
    else:
        data = None
        print("Mat file already exists.")

    print(f"=======Make Masks Soma=======")
    # Extract window size, overlap, and target pixel to micrometer out of input arguments
    window_size_x = window_size_soma[0]
    window_size_y = window_size_soma[1]
    window_size_z = window_size_soma[2]

    overlap_x = overlap[0]
    overlap_y = overlap[1]
    overlap_z = overlap[2]

    # target_px2mu is used to scale in a way to get this ratio
    target_x_px2mu = target_px2mu[0]
    target_y_px2mu = target_px2mu[1]
    target_z_px2mu = target_px2mu[2]

    print(
        f"Target pixel/micrometer: x={target_x_px2mu}, y={target_y_px2mu}, z={target_z_px2mu}")

    if not os.path.exists(save_path_dataset):
        os.makedirs(save_path_dataset)

    # Check cubes flag
    if ignore_flags or (not os.path.exists(flag_path_cubes)):
        img_preprocess.make_mask_soma(
            data = data,
            file_path=os.path.join(
                mat_path,
                dataset_name,
                "dataset.mat"),
            dataset_name=dataset_name,
            save_path_dataset=save_path_dataset,
            window_size_x=window_size_x,
            window_size_y=window_size_y,
            window_size_z=window_size_z,
            overlap_x=overlap_x,
            overlap_y=overlap_y,
            overlap_z=overlap_z,
            target_x_px2mu=target_x_px2mu,
            target_y_px2mu=target_y_px2mu,
            target_z_px2mu=target_z_px2mu,
            num_workers=num_workers)
        # Make file flag_path
        open(flag_path_cubes, 'w').close()
    else:
        print("Cubes already exists.")

    print(f"=======Soma Detection Stage=======")
    print(f"Soma location: {os.path.join(soma_locs, dataset_name)}")
    model_io.soma_detection(
        soma_model_save_path, datasets_path, num_workers, soma_locs)

    # Make file flag_path
    open(flag_path_data, 'w').close()

    print(f"Segmented soma saved in {soma_locs}")

    return soma_locs


def soma2branch_func(branch_model_save_path, save_path, window_size_branch, save_path_soma_locs,
                     kernel_size, branch_segmented_save_path, num_workers, postprocess_flag,
                     ignore_flags, dataset_name) -> Union[bool, str]:
    """ Detect branches after detecting somas

        This function first extract soma locations from the output of soma detection stage,
        and apply the branch detection model for each cell. So, it will produce segmented
        branches for each cell in seperate folders and files.

        Args:
            branch_model_save_path (str): The path to the branch model
            save_path (str): The path to the soma locations
            window_size_branch (tuple): The size of the window for data of branch detection
            save_path_soma_locs (str): The path to save the cubes for branch detection
            kernel_size (int): The size of the kernel for adaptive histogram equalization
            branch_segmented_save_path (str): The path to save the segmented branches
            num_workers (int): The number of workers to use
            postprocess_flag (bool): Whether to apply postprocessing. If True, it will apply dilation, select connected component of the detected soma, erosion, and the return the mask as branches of soma.
            ignore_flags (bool): Whether to ignore flags. If True, it will run the function regardless of previous computations
            dataset_name (str): The name of the dataset
        Returns:
            str: The path to the saved branch segmentation
    """
    # Convert arguments str type to tuple
    window_size_branch = str(window_size_branch)
    window_size_branch = ast.literal_eval(window_size_branch)

    # Set flag paths and save path
    flag_path_branch = os.path.join(
        branch_segmented_save_path, dataset_name, 'flag_branch')
    flag_path_soma = os.path.join(
        save_path_soma_locs, dataset_name, 'flag_soma')
    save_path_soma_locs = os.path.join(
        save_path_soma_locs, dataset_name, 'branches_data')
    branch_segmented_save_path = os.path.join(
        branch_segmented_save_path, dataset_name, 'branch')

    # Check if the branch flag is set. If so, terminate
    if (not ignore_flags) and os.path.exists(flag_path_branch):
        print("IMG to Branch was done already.")
        return True

    if not os.path.exists(save_path_soma_locs):
        os.makedirs(save_path_soma_locs)

    print(f"=======Extract Soma Location=======")
    print(f"------------loading data from: {save_path}")
    print(f"------------saveing data to: {save_path_soma_locs}")

    # Extract window size for branch detection
    window_size_x = window_size_branch[0]
    window_size_y = window_size_branch[1]
    window_size_z = window_size_branch[2]
    print(
        f"window_size_x_branch: {window_size_x} window_size_y_branch: {window_size_y} window_size_z_branch: {window_size_z}")
    
    # Check soma flag for creating cubes for branch detection
    if ignore_flags or (not os.path.exists(flag_path_soma)):
        postprocess.soma_loc_detection(save_path, save_path_soma_locs,
                                       window_size_x, window_size_y, window_size_z,
                                       kernel_size, num_workers)

        # Make the flag for data of branch detection
        open(flag_path_soma, 'w').close()
    else:
        print("Soma location was done already.")

    if not os.path.exists(branch_segmented_save_path):
        os.makedirs(branch_segmented_save_path)

    print(f"======= Branch Segmentation =======")
    print(f"Loading model from {branch_model_save_path}")
    model_io.branch_detection(branch_model_save_path, save_path_soma_locs,
                              branch_segmented_save_path, postprocess_flag)

    # Make the flag for branch detection
    open(flag_path_branch, 'w').close()

    postprocess.reconstruct_img_with_segmented_mask(dataset_name)

    print(f"Branch segmented saved in {branch_segmented_save_path}")

    return True


def branch2swc_func(branch_segmented_save_path) -> Union[bool, str]:
    """ Extract SWC from the segmented branches

        This function converts detected branches to SWC file from the output of soma2branch
        stage. So, it will produce SWC files for each cell separately.

        Args:
            branch_segmented_save_path (str): The path to the segmented branches
        Returns:
            str: The path to save the SWC files
    """
    # Extract SWC
    postprocess.skeleton_extraction(branch_segmented_save_path)

    print(f"SWC saved in {branch_segmented_save_path}")

    return True


def cube2branch_func(cube, point, branch_model_save_path, posprocess_flag) -> Union[bool, str]:
    """ Detect branches for one cube for GUI purpose

        This function is used for GUI purpose to detect branches for one cube. It will
        produce segmented branches for the cube.

        Args:
            cube (np.array): The cube for branch detection
            point (tuple): The point for the cube
            branch_model_save_path (str): The path to the branch model
            posprocess_flag (bool): Whether to apply postprocessing. If True, it will apply dilation, select connected component of the detected soma, erosion, and the return the mask as branches of soma.
        Returns:
            str: The path to the saved branch segmentation
    """
    cube = np.array(cube['arr_0'])
    # remove char in point string
    point = point.replace("'", "")
    point = ast.literal_eval(point)
    print(type(point))
    segmented = model_io.branch_detection_cube(cube, point, branch_model_save_path, posprocess_flag)
    return segmented


@cli.command('img2soma')
@click.option('--img_path', '-ip', type=str, required=True)
@click.option('--soma_model_save_path', '-smsp', type=str, default='/opt/trAIce/trAIce/models/models_best_\soma_2024may07.pth')
@click.option('--soma_locs', '-sl', type=str, required=False, default=None)
@click.option('--window_size_soma', '-wss', type=str, required=True, default="(128,128,16)")
@click.option('--overlap', '-o', type=str, required=True, default="(0,0,0)")
@click.option('--target_px2mu', '-tp', type=str, required=True, default="(2.5,2.5,0.8)")
@click.option('--save_path_dataset', '-spd', type=str, required=True)
@click.option('--mat_path', '-mp', type=str, required=True)
@click.option('--dataset_name', '-dn', type=str, required=False, default=None)
@click.option('--num_workers', '-nw', type=int, required=True, default=1)
@click.option('--soma_model_name', '-smn', type=str, required=True, default="CellSomaSegmentationModel")
@click.option('--ignore_flags', '-if', type=bool, required=True, default=False)
def img2soma(img_path, soma_model_save_path, soma_locs, window_size_soma,
             overlap, target_px2mu, save_path_dataset,
             mat_path, dataset_name, num_workers, soma_model_name, ignore_flags) -> bool:
    """ This function uses appropriate function calling for conversion of each image to detected
        somas.
        Intermediate steps are making cubes for soma detection, feed the model, and save the
        results.

        Args:
            img_path (str): The path to the image file
            soma_model_save_path (str): The path to the soma model
            soma_locs (str): The path to save the segmented somas of soma model, as default save in the dataset folder
            window_size_soma (tuple): The size of the window for soma detection cubes (soma model), default is (128, 128, 16)
            overlap (tuple): The overlap of the window for soma detection cubes (soma model), default is (0, 0, 0)
            target_px2mu (tuple): The target pixel to micron ratio used to scale the image, default is (2.5, 2.5, 0.8)
            save_path_dataset (str): The path to save the cubes for soma detection
            mat_path (str): The path to save the mat file
            dataset_name (str): The name of the dataset, as default is the name of the ims/nd2/tif image file
            num_workers (int): The number of workers to use, default is 1
            soma_model_name (str): The name of the soma detection model in models class
            ignore_flags (bool): Whether to ignore flags. If True, it will run the function regardless of previous computations. default is False
        Returns:
            str: The path to the saved soma locations

        Sample command:
            python main.py img2soma --img_path "test/CX3CR1-EGFP_RETINA_20X.ims" --target_px2mu "(2.5, 2.5, 0.8)" --save_path_dataset "tmp" --mat_path "tmp" --num_workers 4
            -smsp "models/models_best_soma_2024may07.pth"
    """

    # Keep run time
    start = time.time()

    if not img2soma_func(img_path=img_path, soma_model_save_path=soma_model_save_path, soma_locs=soma_locs,
                         window_size_soma=window_size_soma, overlap=overlap, target_px2mu=target_px2mu,
                         save_path_dataset=save_path_dataset, mat_path=mat_path, dataset_name=dataset_name,
                         num_workers=num_workers, ignore_flags=ignore_flags):
        print("Error")

        return False

    end = time.time()
    print(
        f"====> Total Run Time: {int((end - start) / 3600)}h {int((end - start) % 3600 / 60)}m {int((end - start) % 3600 % 60)}s")

    return True


@cli.command('img2branch')
@click.option('--img_path', '-ip', type=str, required=True)
@click.option('--soma_locs', '-sl', type=str, required=False, default=None)
@click.option('--soma_model_save_path', '-smsp', type=str, default='/opt/trAIce/trAIce/models/models_best_\soma_2024may07.pth')
@click.option('--branch_model_save_path', '-bms', type=str, default='/opt/trAIce/trAIce/models/36_UNETPromptGiven_128_128_16/')
@click.option('--window_size_soma', '-wss', type=str, required=True, default="(128, 128, 16)")
@click.option('--window_size_branch', '-wsb', type=str, required=True, default="(512, 512, 32)")
@click.option('--overlap', '-o', type=str, required=True, default="(0, 0, 0)")
@click.option('--target_px2mu', '-tp', type=str, required=True, default=(2.5, 2.5, 0.8))
@click.option('--save_path_dataset', '-spd', type=str, required=True)
@click.option('--mat_path', '-mp', type=str, required=True)
@click.option('--save_path_soma_locs', '-spsl', type=str, required=True)
@click.option('--dataset_name', '-dn', type=str, required=False, default=None)
@click.option('--kernel_size', '-ks', type=int, required=True, default=3)
@click.option('--branch_segmented_save_path', '-bsp', type=str, required=True)
@click.option('--num_workers', '-nw', type=int, required=True)
@click.option('--soma_model_name', '-smn', type=str, required=True, default="CellSomaSegmentationModel")
@click.option('--postprocess_flag', '-ppf', type=bool, required=True, default=False)
@click.option('--ignore_flags', '-if', type=bool, required=True, default=False)
def img2branch(img_path, soma_model_save_path, soma_locs, branch_model_save_path,
               window_size_soma, window_size_branch, overlap, target_px2mu,
               save_path_dataset, mat_path, dataset_name, save_path_soma_locs,
               kernel_size, branch_segmented_save_path, num_workers, soma_model_name,
               postprocess_flag, ignore_flags) -> bool:
    """ This function uses appropriate function calling for conversion of each image to branches.
        Intermediate steps are making cubes for soma detection, feed the model, and save the
        results for soma detection, extract soma locations, feed the branch detection model,
        save the results for branch detection.

        Args:
            img_path (str): The path to the image file
            soma_model_save_path (str): The path to the soma model
            soma_locs (str): The path to save the segmented somas of soma model, as default save in the dataset folder
            window_size_soma (tuple): The size of the window for soma detection cubes (soma model), default is (128, 128, 16)
            overlap (tuple): The overlap of the window for soma detection cubes (soma model), default is (0, 0, 0)
            target_px2mu (tuple): The target pixel to micron ratio used to scale the image, default is (2.5, 2.5, 0.8)
            save_path_dataset (str): The path to save the cubes for soma detection
            mat_path (str): The path to save the mat file
            dataset_name (str): The name of the dataset, as default is the name of the ims/nd2/tif image file
            num_workers (int): The number of workers to use, default is 1
            soma_model_name (str): The name of the soma detection model in models class
            ignore_flags (bool): Whether to ignore flags. If True, it will run the function regardless of previous computations. default is False
            branch_model_save_path (str): The path to the branch model
            window_size_branch (tuple): The size of the cubes for data of branch detection, default is (512, 512, 32)
            save_path_soma_locs (str): The path to save the cubes for branch detection
            kernel_size (int): The size of the kernel for adaptive histogram equalization, default is 3
        Returns:
            str: The path to the saved branch segmentation
    """

    # Keep run time
    start = time.time()

    if not img2soma_func(img_path=img_path, soma_model_save_path=soma_model_save_path, soma_locs=soma_locs,
                         window_size_soma=window_size_soma, overlap=overlap, target_px2mu=target_px2mu,
                         save_path_dataset=save_path_dataset, mat_path=mat_path, dataset_name=dataset_name,
                         num_workers=num_workers, ignore_flags=ignore_flags):
        print("Error")
        return False

    if not soma2branch_func(branch_model_save_path=branch_model_save_path,
                            save_path=[os.path.join(save_path_dataset,
                                                    [os.path.basename(img_path).split(".")[0] if dataset_name is None else dataset_name][0],
                                                    'soma') if soma_locs is None else soma_locs][0],
                            window_size_branch=window_size_branch,
                            save_path_soma_locs=save_path_soma_locs,
                            dataset_name=os.path.basename(img_path).split(".")[0],
                            kernel_size=kernel_size,
                            branch_segmented_save_path=branch_segmented_save_path,
                            num_workers=num_workers,
                            postprocess_flag=postprocess_flag,
                            ignore_flags=ignore_flags):
        print("Error")
        return False

    end = time.time()
    print(
        f"====> Total Run Time: {int((end - start) / 3600)}h {int((end - start) % 3600 / 60)}m {int((end - start) % 3600 % 60)}s")

    return True


@cli.command('img2swc')
@click.option('--img_path', '-ip', type=str, required=True)
@click.option('--soma_model_save_path', '-smsp', type=str, default='/opt/trAIce/trAIce/models/models_best_\soma_2024may07.pth')
@click.option('--branch_model_save_path', '-bms', type=str, default='/opt/trAIce/trAIce/models/36_UNETPromptGiven_128_128_16/')
@click.option('--window_size_soma', '-wss', type=str, required=True, default="(128, 128, 16)")
@click.option('--window_size_branch', '-wsb', type=str, required=True, default="(128, 128, 16)")
@click.option('--overlap', '-o', type=str, required=True, default="(0, 0, 0)")
@click.option('--target_px2mu', '-tp', type=str, required=True, default="(2.5, 2.5, 0.8)")
@click.option('--save_path_dataset', '-spd', type=str, required=True)
@click.option('--mat_path', '-mp', type=str, required=True)
@click.option('--save_path_soma_locs', '-spsl', type=str, required=True)
@click.option('--dataset_name', '-dn', type=str, required=False, default=None)
@click.option('--soma_locs', '-sl', type=str, required=False, default=None)
@click.option('--kernel_size', '-ks', type=int, required=True, default=3)
@click.option('--branch_segmented_save_path', '-bsp', type=str, required=True)
@click.option('--num_workers', '-nw', type=int, required=True)
@click.option('--ignore_flags', '-if', type=bool, required=True, default=False)
@click.option('--soma_model_name', '-smn', type=str, required=True, default="CellSomaSegmentationModel")
@click.option('--postprocess_flag', '-ppf', type=bool, required=True, default=False)
@click.option('--ignore_flags', '-if', type=bool, required=True, default=False)
def img2swc(img_path, soma_model_save_path, branch_model_save_path,
            window_size_soma, window_size_branch, soma_locs,
            overlap, target_px2mu, save_path_dataset,
            mat_path, dataset_name, soma_model_name,
            save_path_soma_locs, kernel_size, postprocess_flag,
            branch_segmented_save_path, num_workers, ignore_flags) -> bool:
    """ This function uses appropriate function calling for conversion of each image to SWC files.
        Intermediate steps are making cubes for soma detection, feed the model, and save the
        results for soma detection, extract soma locations, feed the branch detection model,
        save the results for branch detection, and extract SWC files.

        Args:
            img_path (str): The path to the image file
            soma_model_save_path (str): The path to the soma model
            soma_locs (str): The path to save the segmented somas of soma model, as default save in the dataset folder
            window_size_soma (tuple): The size of the window for soma detection cubes (soma model), default is (128, 128, 16)
            overlap (tuple): The overlap of the window for soma detection cubes (soma model), default is (0, 0, 0)
            target_px2mu (tuple): The target pixel to micron ratio used to scale the image, default is (2.5, 2.5, 0.8)
            save_path_dataset (str): The path to save the cubes for soma detection
            mat_path (str): The path to save the mat file
            dataset_name (str): The name of the dataset, as default is the name of the ims/nd2/tif image file
            num_workers (int): The number of workers to use, default is 1
            soma_model_name (str): The name of the soma detection model in models class
            ignore_flags (bool): Whether to ignore flags. If True, it will run the function regardless of previous computations. default is False
            branch_model_save_path (str): The path to the branch model
            window_size_branch (tuple): The size of the cubes for data of branch detection, default is (512, 512, 32)
            save_path_soma_locs (str): The path to save the cubes for branch detection
            kernel_size (int): The size of the kernel for adaptive histogram equalization, default is 3
            postprocess_flag (bool): Whether to apply postprocessing. If True, it will apply dilation, select connected component of the detected soma, erosion, and the return the mask as branches of soma. default is False
            branch_segmented_save_path (str): The path to save SWC files
        Returns:
            str: The path to the saved SWC files
    """

    # Keep run time
    start = time.time()

    if not img2soma_func(img_path=img_path, soma_model_save_path=soma_model_save_path, soma_locs=soma_locs,
                         window_size_soma=window_size_soma, overlap=overlap, target_px2mu=target_px2mu,
                         save_path_dataset=save_path_dataset, mat_path=mat_path, dataset_name=dataset_name,
                         num_workers=num_workers, ignore_flags=ignore_flags):
        print("Error")
        return False

    if not soma2branch_func(branch_model_save_path=branch_model_save_path,
                            save_path=[os.path.join(save_path_dataset,
                                                    [os.path.basename(img_path).split(".")[0] if dataset_name is None else dataset_name][0],
                                                    'soma') if soma_locs is None else soma_locs][0],
                            window_size_branch=window_size_branch,
                            save_path_soma_locs=save_path_soma_locs,
                            dataset_name=os.path.basename(img_path).split(".")[0],
                            kernel_size=kernel_size,
                            branch_segmented_save_path=branch_segmented_save_path,
                            num_workers=num_workers,
                            postprocess_flag=postprocess_flag,
                            ignore_flags=ignore_flags):
        print("Error")
        return False

    # print(f"branch_segmented_save_path {branch_segmented_save_path}")
    if not branch2swc_func(
        branch_segmented_save_path=os.path.join(
            branch_segmented_save_path, [
            os.path.basename(img_path).split(".")[0] if dataset_name is None else dataset_name][0], 'branch')):
        print("Error")
        return False

    end = time.time()
    print(
        f"====> Total Run Time: {int((end - start) / 3600)}h {int((end - start) % 3600 / 60)}m {int((end - start) % 3600 % 60)}s")

    return True


if __name__ == '__main__':
    cli()
